<a href="https://colab.research.google.com/github/simon-air/work/blob/master/ttjj_fee_r0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
#base_dir = root_dir + 'DataSet/zycg20190606/'
base_dir = root_dir + '/Quant/'

Mounted at /content/gdrive


In [0]:
import re
import requests
import json
import pandas as pd
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
import time
import random

In [0]:
r = requests.get('http://fund.eastmoney.com/js/fundcode_search.js')
cont = re.findall('var r = (.*])', r.text)[0]  # 提取list
ls = json.loads(cont)  # 将字符串格式的list转化为list格式
all_fundCode = pd.DataFrame(ls, columns=['基金代码', '基金名称缩写', '基金名称', '基金类型', '基金名称拼音'])  # list转为DataFrame
all_fundCode = all_fundCode.loc[:,['基金代码','基金名称','基金类型']]
all_fundCode.head(5)

,基金代码,基金名称,基金类型
0,000001,华夏成长,混合型
1,000002,华夏成长(后端),混合型
2,000003,中海可转债债券A,债券型
3,000004,中海可转债债券C,债券型
4,000005,嘉实增强信用定期债券,定开债券


In [0]:
all_fundCode.to_csv('/content/gdrive/My Drive/Quant/all_fundCode.csv', encoding='utf_8_sig',index=False)

In [0]:
gpzs_fundCode = all_fundCode.loc[all_fundCode['基金类型'].isin(['股票指数','ETF-场内','QDII-ETF','QDII-指数','联接基金','债券指数'])]
len(gpzs_fundCode)

1132

In [0]:
gpzs_fundCode.head(5)

,基金代码,基金名称,基金类型
6,000008,嘉实中证500ETF联接A,联接基金
30,000042,中证财通可持续发展100指数A,股票指数
37,000049,中银标普全球资源等权重指数,QDII-指数
38,000051,华夏沪深300ETF联接A,联接基金
41,000055,广发纳斯达克100美元现汇A,QDII-指数


In [0]:
gpzs_fundCode.to_csv('/content/gdrive/My Drive/Quant/gpzs_fundCode.csv', encoding='utf_8_sig',index=False)

In [0]:
'''
fundCode = '006743'
jbgk_url = 'http://fundf10.eastmoney.com/jbgk_'+ fundCode + '.html'
soup = BeautifulSoup(urllib.request.urlopen(jbgk_url))
'''

In [0]:
'''
def parse_one_page(soup):
    glf = soup.find('th',text="管理费率")
    glfl = glf.find_next_sibling('td').string
    tgf = soup.find('th',text="托管费率")
    tgfl = tgf.find_next_sibling('td').string
    xsf = soup.find('th',text="销售服务费率")
    xsfl = xsf.find_next_sibling('td').string
    zcg = soup.find('th',text="资产规模")
    zcgm = zcg.find_next_sibling('td').string
    gzb = soup.find('th',text="跟踪标的")
    gzbd = gzb.find_next_sibling('td').string
 '''

In [0]:
user_agent_list = [
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER',
        'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)',
        'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 SE 2.X MetaSr 1.0',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Maxthon/4.4.3.4000 Chrome/30.0.1599.101 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.122 UBrowser/4.0.3214.0 Safari/537.36',]

In [0]:
#GPU,执行时长494秒
DF = []
for fundCode in gpzs_fundCode['基金代码']:
    jbgk_url = 'http://fundf10.eastmoney.com/jbgk_'+ fundCode + '.html'
    req = urllib.request.Request(jbgk_url,headers = {'User-Agent': random.choice(user_agent_list)})
    response = urllib.request.urlopen(req)
    soup = BeautifulSoup(response)
    jjj = soup.find('th',text="基金简称")
    jjjc = jjj.find_next_sibling('td').string
    fxr = soup.find('th',text="发行日期")
    fxrq = fxr.find_next_sibling('td').string
    jjg = soup.find('th',text="基金管理人")
    jjgl = jjg.find_next_sibling('td').string
    glf = soup.find('th',text="管理费率")
    glfl = glf.find_next_sibling('td').string
    tgf = soup.find('th',text="托管费率")
    tgfl = tgf.find_next_sibling('td').string
    xsf = soup.find('th',text="销售服务费率")
    xsfl = xsf.find_next_sibling('td').string
    zcg = soup.find('th',text="资产规模")
    zcgm = zcg.find_next_sibling('td').string
    gzb = soup.find('th',text="跟踪标的")
    gzbd = gzb.find_next_sibling('td').string
    tsdata_url = 'http://fundf10.eastmoney.com/tsdata_'+ fundCode + '.html'
    req = urllib.request.Request(tsdata_url,headers = {'User-Agent': random.choice(user_agent_list)})
    response = urllib.request.urlopen(req)
    soup = BeautifulSoup(response)
    gzw = soup.find('th',text="跟踪误差")
    if hasattr(gzw,'parent'):
        gzwc = gzw.parent.next_sibling.next_sibling.find_all('td')[1].string
    else: gzwc = 'n/a'
    DF.append([fundCode,jjjc,fxrq,jjgl,glfl,tgfl,xsfl,zcgm,gzbd,gzwc])
#    time.sleep(random.randint(5, 10))

In [0]:
    '''
    tsdata_url = 'http://fundf10.eastmoney.com/tsdata_'+ '000950' + '.html'
    req = urllib.request.Request(tsdata_url,headers = {'User-Agent': random.choice(user_agent_list)})
    response = urllib.request.urlopen(req)
    soup = BeautifulSoup(response)
    gzw = soup.find('th',text="跟踪误差")
    gzwc = gz.parent.next_sibling.next_sibling.find_all('td')[1].string
    
    '''

In [0]:
data=pd.DataFrame(DF,columns=['基金代码',"基金简称","发行日期","基金管理人",'管理费','托管费','销售服务费','资产规模','跟踪标的',"跟踪误差"])
print(data)
data.to_csv('/content/gdrive/My Drive/Quant/gpzs_fundCode_r0.1.csv', encoding='utf_8_sig',index=False)

        基金代码             基金简称  ...                           跟踪标的   跟踪误差
0     000008    嘉实中证500ETF联接A  ...                        中证500指数  0.10%
1     000042  中证财通可持续发展100指数A  ...     中证财通中国可持续发展100(ECPI ESG)指数  0.27%
2     000049    中银标普全球资源等权重指数  ...                标普全球精选自然资源等权重指数     --
3     000051    华夏沪深300ETF联接A  ...                        沪深300指数  0.10%
4     000055   广发纳斯达克100美元现汇A  ...                      纳斯达克100指数  0.24%
5     000059      国联安中证医药100A  ...                      中证医药100指数  0.19%
6     000071       华夏恒生ETF联接A  ...                         香港恒生指数  0.23%
7     000075      华夏恒生ETF联接现汇  ...                         香港恒生指数  0.23%
8     000076      华夏恒生ETF联接现钞  ...                         香港恒生指数  0.23%
9     000087   嘉实中证金边国债ETF联接A  ...                     中证金边中期国债指数  0.08%
10    000088   嘉实中证金边国债ETF联接C  ...                     中证金边中期国债指数  0.08%
11    000154      富国沪深300指数增强  ...                        沪深300指数    n/a
12    000164         富国上证综指联接  ...                 